In [4]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm = llm,
    max_token_limit=120,
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpul AI talking to a human"),
    MessagesPlaceholder(variable_name="history"),
    ("human","{question}"),
])

def load_memory(_):
    return memory.load_memory_variables({})["history"]

# RunnablePassthrough.assign() : 프롬프트가 format되기 전에 우리가 함수를 실행시키는걸 변수에 담아 다음 프롬프트에 주어진다.
chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
  result = chain.invoke({
    "question":question
  })
  memory.save_context({"input":question}, {"output":result.content})
  print(result)


In [5]:
invoke_chain("My name is Justddan")

content='Hello Justddan! How can I assist you today?'
